In [ ]:
# default_exp core

# flowsy

> API details.

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#export
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'

Use this to say hello to anyone

In [ ]:
say_hello("Daniel")

'Hello Daniel!'

In [ ]:
test_eq(say_hello("Daniel"), "Hello Daniel!")

In [ ]:
#export
class HelloSayer:
    "Say hello to `to` using `say_hello`"
    def __init__(self, to): self.to = to
    def say(self): return say_hello(self.to)

In [ ]:
show_doc(HelloSayer.say)

<h4 id="HelloSayer.say" class="doc_header"><code>HelloSayer.say</code><a href="__main__.py#L4" class="source_link" style="float:right">[source]</a></h4>

> <code>HelloSayer.say</code>()



In [ ]:
o = HelloSayer("Alexis")
o.say()

'Hello Alexis!'

In [ ]:
#export
def dummy(file='Hallo'):
    return file